# IMDB 영화 리뷰 감성 분석 - 완전 가이드

## 📚 학습 목표
1. **자연어 처리(NLP) 기초 이해**: 텍스트 데이터를 머신러닝에 활용하는 방법 학습
2. **실제 데이터 파이프라인 구축**: 대용량 텍스트 데이터 전처리부터 모델 배포까지
3. **감성 분석 모델 개발**: 이진 분류를 통한 감정 예측 시스템 구축
4. **성능 최적화 기법**: 데이터 캐싱, 벡터화, 모델 튜닝 기법 적용

## 🎯 프로젝트 개요
- **데이터셋**: Stanford AI Lab의 IMDB 영화 리뷰 (50,000개 리뷰)
- **문제 유형**: 이진 분류 (긍정/부정 감성 분석)
- **기술 스택**: TensorFlow/Keras, TextVectorization, Dense Neural Network
- **벡터화 방식**: Multi-hot encoding (Bag of Words)

## 📋 노트북 구성
```
Part I   : 이론 및 배경 지식
Part II  : 환경 설정 및 데이터 준비  
Part III : 텍스트 벡터화 및 전처리
Part IV  : 모델 구축 및 훈련
Part V   : 성능 평가 및 개선 방안
```


# Part I: 이론 및 배경 지식

## 1.1 감성 분석이란?

**감성 분석(Sentiment Analysis)**은 텍스트에 담긴 감정, 의견, 태도를 자동으로 분석하는 자연어 처리 기법입니다.

### 🔍 주요 응용 분야
- **소셜 미디어 모니터링**: 브랜드에 대한 대중의 반응 분석
- **제품 리뷰 분석**: 고객 만족도 및 피드백 자동 분류
- **주식 시장 예측**: 뉴스 기사의 감성을 통한 시장 동향 예측
- **고객 서비스**: 문의 내용의 긴급도 및 감정 상태 파악

### 📊 분류 유형
1. **이진 분류**: 긍정/부정 (본 프로젝트)
2. **다중 분류**: 매우 긍정/긍정/중립/부정/매우 부정
3. **세밀한 감정 분류**: 기쁨, 슬픔, 분노, 두려움 등

## 1.2 IMDB 데이터셋 소개

### 📈 데이터셋 특징
- **출처**: Stanford AI Lab (Andrew Maas et al., 2011)
- **규모**: 총 50,000개 영화 리뷰
- **구성**: 
  - 훈련용: 25,000개 (긍정 12,500 + 부정 12,500)
  - 테스트용: 25,000개 (긍정 12,500 + 부정 12,500)
- **레이블**: 이진 분류 (0: 부정, 1: 긍정)
- **크기**: 압축 파일 ~80MB, 압축 해제 후 ~280MB

### 🎭 데이터 품질
- **균형 잡힌 데이터**: 긍정/부정 비율이 50:50으로 균등
- **실제 사용자 리뷰**: IMDb 웹사이트의 실제 사용자 작성 리뷰
- **품질 필터링**: 극단적 점수(1-4점: 부정, 7-10점: 긍정)만 포함

## 1.3 텍스트 벡터화 이론

### 🧮 왜 벡터화가 필요한가?
머신러닝 모델은 **숫자**만 처리할 수 있습니다. 따라서 텍스트를 숫자로 변환하는 과정이 필수입니다.

### 🎯 주요 벡터화 기법

#### 1) Bag of Words (BoW) - 본 프로젝트에서 사용
```
"I love this movie" → [0, 1, 1, 0, 1, 0, ...]
```
- **장점**: 구현 간단, 해석 용이
- **단점**: 단어 순서 무시, 희소 벡터

#### 2) TF-IDF (Term Frequency-Inverse Document Frequency)
```
TF-IDF = TF(단어 빈도) × IDF(역문서 빈도)
```
- **장점**: 중요한 단어에 가중치 부여
- **단점**: 여전히 순서 정보 없음

#### 3) Word Embeddings (Word2Vec, GloVe)
```
"king" - "man" + "woman" ≈ "queen"
```
- **장점**: 의미론적 관계 표현
- **단점**: 사전 훈련 필요

#### 4) Transformer 기반 (BERT, GPT)
- **장점**: 문맥 고려, 최고 성능
- **단점**: 계산 비용 높음


In [51]:
# Part II: 환경 설정 및 데이터 준비

# 필수 라이브러리 임포트
import requests          # HTTP 요청 (데이터 다운로드)
import subprocess        # 시스템 명령어 실행 (압축 해제)
import re, string        # 정규표현식, 문자열 처리
import os, pathlib       # 파일/디렉토리 관리
import shutil, random    # 파일 이동, 랜덤 처리

# TensorFlow & Keras (딥러닝 프레임워크)
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import keras
from keras import layers, models

# 추가 유틸리티
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# TensorFlow 버전 확인 및 설정
print("🔧 환경 설정")
print("="*50)
print(f"TensorFlow 버전: {tf.__version__}")
print(f"Keras 버전: {keras.__version__}")
print(f"Python 실행 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# GPU 사용 가능 여부 확인
if tf.config.list_physical_devices('GPU'):
    print("✅ GPU 사용 가능")
    print(f"GPU 장치: {tf.config.list_physical_devices('GPU')}")
else:
    print("⚠️ CPU만 사용 (GPU 없음)")

# 재현 가능한 결과를 위한 시드 설정
RANDOM_SEED = 1337
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
print(f"🎲 랜덤 시드 설정: {RANDOM_SEED}")

print("\n✅ 환경 설정 완료!")


🔧 환경 설정
TensorFlow 버전: 2.15.1
Keras 버전: 2.15.0
Python 실행 시간: 2025-07-31 15:59:09
⚠️ CPU만 사용 (GPU 없음)
🎲 랜덤 시드 설정: 1337

✅ 환경 설정 완료!


## 2.1 데이터 다운로드 시스템

### 🎯 다운로드 전략
- **스트리밍 방식**: 메모리 효율적인 대용량 파일 처리
- **진행률 표시**: 사용자 경험 개선
- **오류 처리**: 네트워크 문제 대응
- **중복 다운로드 방지**: 기존 파일 존재 시 건너뛰기

### 📊 파일 정보
- **URL**: `https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz`
- **크기**: 약 80MB (압축), 280MB (압축 해제)
- **형식**: tar.gz (Linux 표준 압축 형식)


In [52]:
def download_imdb_dataset():
    """
    IMDB 데이터셋을 안전하고 효율적으로 다운로드하는 함수
    
    Features:
    - 스트리밍 다운로드로 메모리 효율성 확보
    - 진행률 표시 및 속도 측정
    - 기존 파일 검증 및 중복 다운로드 방지
    - 네트워크 오류 처리
    
    Returns:
        bool: 다운로드 성공 여부
    """
    
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    file_name = "../../data/aclImdb_v1.tar.gz"
    
    # 디렉토리 생성
    os.makedirs("../../data", exist_ok=True)
    
    # 기존 파일 확인
    if os.path.exists(file_name):
        file_size = os.path.getsize(file_name) / (1024 * 1024)  # MB 단위
        print(f"✅ 파일이 이미 존재합니다: {file_name}")
        print(f"📊 파일 크기: {file_size:.1f}MB")
        return True
    
    print(f"📥 데이터셋 다운로드 시작...")
    print(f"🔗 URL: {url}")
    print(f"📁 저장 위치: {file_name}")
    
    try:
        # HTTP 요청 시작
        response = requests.get(url, stream=True, timeout=30)
        response.raise_for_status()  # HTTP 에러 확인
        
        # 파일 크기 확인
        total_size = int(response.headers.get('content-length', 0))
        downloaded = 0
        chunk_size = 8192  # 8KB
        
        print(f"📊 총 파일 크기: {total_size / (1024*1024):.1f}MB")
        
        # 파일 다운로드
        with open(file_name, "wb") as file:
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:  # 빈 청크 필터링
                    file.write(chunk)
                    downloaded += len(chunk)
                    
                    # 진행률 표시 (매 1MB마다)
                    if downloaded % (1024 * 1024) < chunk_size:
                        progress = (downloaded / total_size) * 100 if total_size else 0
                        print(f"⏬ 다운로드 진행률: {progress:.1f}% ({downloaded/(1024*1024):.1f}MB)")
        
        print(f"✅ 다운로드 완료!")
        print(f"📁 저장된 파일: {file_name}")
        return True
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 네트워크 오류: {e}")
        return False
    except Exception as e:
        print(f"❌ 다운로드 중 오류 발생: {e}")
        return False

# 다운로드 실행 (필요시 주석 해제)
# success = download_imdb_dataset()
# if success:
#     print("🎉 데이터 준비 완료!")
# else:
#     print("💥 데이터 다운로드 실패!")


## 2.2 압축 해제 및 파일 구조 분석

### 🗂️ tar.gz 형식 이해
- **tar**: 여러 파일을 하나로 묶는 아카이브 형식
- **gz**: gzip 압축 알고리즘 적용
- **Linux/Unix 표준**: 소스 코드 배포에 널리 사용

### 📁 예상 디렉토리 구조
```
aclImdb/
├── train/
│   ├── pos/          # 긍정 리뷰 (12,500개)
│   ├── neg/          # 부정 리뷰 (12,500개)
│   └── unsup/        # 무레이블 데이터 (제거 예정)
├── test/
│   ├── pos/          # 긍정 리뷰 (12,500개)
│   └── neg/          # 부정 리뷰 (12,500개)
└── README           # 데이터셋 설명
```


In [53]:
def extract_imdb_dataset():
    """
    IMDB 데이터셋 압축 파일을 안전하게 해제하는 함수
    
    Features:
    - 크로스 플랫폼 지원 (Python tarfile 모듈 사용)
    - 진행률 표시 및 상태 모니터링
    - 기존 디렉토리 검증
    - 파일 구조 검증
    
    Returns:
        bool: 압축 해제 성공 여부
    """
    
    import tarfile
    
    archive_path = "../../data/aclImdb_v1.tar.gz"
    extract_path = "../../data/"
    target_dir = "../../data/aclImdb"
    
    # 이미 해제된 디렉토리 확인
    if os.path.exists(target_dir):
        print(f"✅ 이미 압축 해제된 디렉토리가 존재합니다: {target_dir}")
        
        # 디렉토리 구조 검증
        required_dirs = ['train/pos', 'train/neg', 'test/pos', 'test/neg']
        missing_dirs = []
        
        for dir_path in required_dirs:
            full_path = os.path.join(target_dir, dir_path)
            if not os.path.exists(full_path):
                missing_dirs.append(dir_path)
        
        if missing_dirs:
            print(f"⚠️ 누락된 디렉토리: {missing_dirs}")
            print("🔄 재해제를 진행합니다...")
        else:
            print("✅ 디렉토리 구조 검증 완료!")
            return True
    
    # 압축 파일 존재 확인
    if not os.path.exists(archive_path):
        print(f"❌ 압축 파일을 찾을 수 없습니다: {archive_path}")
        print("💡 먼저 다운로드를 실행해주세요.")
        return False
    
    print(f"📦 압축 해제 시작...")
    print(f"🗂️ 압축 파일: {archive_path}")
    print(f"📁 해제 위치: {extract_path}")
    
    try:
        # tar 파일 열기 및 검증
        with tarfile.open(archive_path, 'r:gz') as tar:
            # 파일 목록 확인
            members = tar.getmembers()
            total_files = len(members)
            
            print(f"📊 총 파일 수: {total_files:,}개")
            print(f"⏳ 압축 해제 중... (시간이 소요될 수 있습니다)")
            
            # 안전한 압축 해제 (경로 검증)
            def safe_extract(tarinfo, path):
                # 경로 traversal 공격 방지
                if os.path.isabs(tarinfo.name) or ".." in tarinfo.name:
                    print(f"⚠️ 안전하지 않은 경로 건너뛰기: {tarinfo.name}")
                    return None
                return tarinfo
            
            # 진행률 표시와 함께 해제
            extracted_count = 0
            for member in members:
                if safe_extract(member, extract_path):
                    tar.extract(member, extract_path)
                    extracted_count += 1
                    
                    # 진행률 표시 (매 1000개 파일마다)
                    if extracted_count % 1000 == 0:
                        progress = (extracted_count / total_files) * 100
                        print(f"⏬ 압축 해제 진행률: {progress:.1f}% ({extracted_count:,}/{total_files:,})")
        
        print(f"✅ 압축 해제 완료!")
        
        # 해제 결과 검증
        if os.path.exists(target_dir):
            print(f"📁 생성된 디렉토리: {target_dir}")
            
            # 각 하위 디렉토리의 파일 수 확인
            subdirs = ['train/pos', 'train/neg', 'test/pos', 'test/neg']
            for subdir in subdirs:
                subdir_path = os.path.join(target_dir, subdir)
                if os.path.exists(subdir_path):
                    file_count = len([f for f in os.listdir(subdir_path) if f.endswith('.txt')])
                    print(f"  📂 {subdir}: {file_count:,}개 파일")
            
            return True
        else:
            print(f"❌ 대상 디렉토리가 생성되지 않았습니다.")
            return False
            
    except tarfile.TarError as e:
        print(f"❌ tar 파일 오류: {e}")
        return False
    except Exception as e:
        print(f"❌ 압축 해제 중 오류 발생: {e}")
        return False

# 압축 해제 실행 (필요시 주석 해제)
# success = extract_imdb_dataset()
# if success:
#     print("🎉 데이터 압축 해제 완료!")
# else:
#     print("💥 압축 해제 실패!")


## 2.3 데이터 분할 전략

### 🎯 분할의 중요성
머신러닝에서 데이터 분할은 모델의 신뢰성 있는 평가를 위해 필수적입니다.

### 📊 분할 전략
```
원본 데이터: train(25,000) + test(25,000)
        ↓
최종 구조: train(20,000) + val(5,000) + test(25,000)
```

### 🔄 분할 과정
1. **Train 데이터 분할**: 25,000개 → 20,000개(훈련) + 5,000개(검증)
2. **무작위 샘플링**: 편향 방지를 위한 랜덤 셔플
3. **클래스 균형 유지**: 긍정/부정 비율 50:50 유지
4. **Unsupervised 데이터 제거**: 레이블이 없는 데이터 삭제

### 💡 분할 비율 선택 이유
- **Train 80%**: 충분한 학습 데이터 확보
- **Validation 20%**: 신뢰할 만한 성능 추정
- **Test**: 완전 독립적 최종 평가


In [54]:
def create_validation_split(validation_ratio=0.2, random_seed=1337):
    """
    훈련 데이터를 train/validation으로 분할하는 함수
    
    Args:
        validation_ratio (float): 검증 데이터 비율 (기본: 20%)
        random_seed (int): 재현 가능한 결과를 위한 시드
    
    Features:
    - 클래스별 균등 분할 (긍정/부정 비율 유지)
    - 기존 분할 검증 및 건너뛰기
    - 상세한 분할 통계 제공
    - 무레이블 데이터 자동 제거
    
    Returns:
        bool: 분할 성공 여부
    """
    
    base_dir = pathlib.Path("../../data/aclImdb")
    train_dir = base_dir / "train"
    val_dir = base_dir / "val"
    
    # 기본 디렉토리 존재 확인
    if not base_dir.exists():
        print(f"❌ 데이터셋 디렉토리가 존재하지 않습니다: {base_dir}")
        print("💡 먼저 데이터 다운로드 및 압축 해제를 실행해주세요.")
        return False
    
    # 이미 분할이 완료된 경우 확인
    if val_dir.exists():
        print(f"✅ Validation 디렉토리가 이미 존재합니다: {val_dir}")
        
        # 분할 상태 검증
        categories = ['pos', 'neg']
        split_valid = True
        
        for category in categories:
            train_count = len(list((train_dir / category).glob('*.txt'))) if (train_dir / category).exists() else 0
            val_count = len(list((val_dir / category).glob('*.txt'))) if (val_dir / category).exists() else 0
            
            print(f"  📂 {category}: train({train_count:,}) + val({val_count:,}) = {train_count + val_count:,}")
            
            if train_count == 0 or val_count == 0:
                split_valid = False
        
        if split_valid:
            print("✅ 데이터 분할이 이미 완료되어 있습니다!")
            return True
        else:
            print("⚠️ 분할 상태가 불완전합니다. 재분할을 진행합니다...")
    
    print(f"🔄 데이터 분할 시작")
    print(f"📊 분할 비율: Train {(1-validation_ratio)*100:.0f}% | Validation {validation_ratio*100:.0f}%")
    print(f"🎲 랜덤 시드: {random_seed}")
    
    try:
        # Validation 디렉토리 생성
        val_dir.mkdir(exist_ok=True)
        
        # 무레이블 데이터 제거
        unsup_dir = train_dir / "unsup"
        if unsup_dir.exists():
            print(f"🗑️ 무레이블 데이터 제거 중: {unsup_dir}")
            shutil.rmtree(unsup_dir)
            print("✅ 무레이블 데이터 제거 완료")
        
        total_moved = 0
        
        # 각 카테고리별 분할 처리
        for category in ["neg", "pos"]:
            print(f"\n📂 {category.upper()} 카테고리 처리 중...")
            
            # 디렉토리 생성
            train_category_dir = train_dir / category
            val_category_dir = val_dir / category
            val_category_dir.mkdir(exist_ok=True)
            
            # 파일 목록 가져오기
            if not train_category_dir.exists():
                print(f"⚠️ 카테고리 디렉토리가 존재하지 않습니다: {train_category_dir}")
                continue
                
            files = [f for f in train_category_dir.glob('*.txt')]
            original_count = len(files)
            
            print(f"  📊 원본 파일 수: {original_count:,}개")
            
            # 파일 목록 랜덤 셔플
            rng = random.Random(random_seed)
            rng.shuffle(files)
            
            # Validation용 파일 수 계산
            num_val_samples = int(validation_ratio * len(files))
            val_files = files[:num_val_samples]  # 처음 20%를 validation으로
            
            print(f"  ⬇️ Validation으로 이동: {num_val_samples:,}개")
            print(f"  📋 Train에 유지: {len(files) - num_val_samples:,}개")
            
            # 파일 이동
            moved_count = 0
            failed_count = 0
            
            for file_path in val_files:
                try:
                    dest_path = val_category_dir / file_path.name
                    shutil.move(str(file_path), str(dest_path))
                    moved_count += 1
                except Exception as e:
                    print(f"  ⚠️ 파일 이동 실패 {file_path.name}: {e}")
                    failed_count += 1
            
            print(f"  ✅ 성공적으로 이동: {moved_count:,}개")
            if failed_count > 0:
                print(f"  ❌ 이동 실패: {failed_count:,}개")
            
            total_moved += moved_count
        
        print(f"\n🎉 데이터 분할 완료!")
        print(f"📊 총 이동된 파일: {total_moved:,}개")
        
        # 최종 분할 결과 확인
        print(f"\n📈 최종 분할 결과:")
        print("=" * 40)
        
        for split_name in ['train', 'val']:
            split_dir = base_dir / split_name
            if split_dir.exists():
                pos_count = len(list((split_dir / 'pos').glob('*.txt')))
                neg_count = len(list((split_dir / 'neg').glob('*.txt')))
                total_count = pos_count + neg_count
                
                print(f"{split_name.upper():>5}: 긍정 {pos_count:,} + 부정 {neg_count:,} = 총 {total_count:,}개")
        
        return True
        
    except Exception as e:
        print(f"❌ 데이터 분할 중 오류 발생: {e}")
        return False

# 데이터 분할 실행 (필요시 주석 해제)
# success = create_validation_split()
# if success:
#     print("🎉 데이터 분할 완료!")
# else:
#     print("💥 데이터 분할 실패!")

# Part III: 텍스트 벡터화 및 전처리

## 3.1 데이터셋 로딩 시스템

### 🎯 Keras Dataset API 활용
`text_dataset_from_directory`는 디렉토리 구조에서 자동으로 텍스트 분류 데이터셋을 생성하는 강력한 도구입니다.

### 📁 예상 디렉토리 구조
```
../../data/aclImdb/
├── train/
│   ├── pos/        # 긍정 리뷰 → 라벨 1
│   └── neg/        # 부정 리뷰 → 라벨 0
├── val/
│   ├── pos/        # 긍정 리뷰 → 라벨 1  
│   └── neg/        # 부정 리뷰 → 라벨 0
└── test/
    ├── pos/        # 긍정 리뷰 → 라벨 1
    └── neg/        # 부정 리뷰 → 라벨 0
```

### ⚙️ 주요 설정 파라미터
- **batch_size**: 한 번에 처리할 샘플 수 (메모리 vs 학습 속도 트레이드오프)
- **shuffle**: 훈련 데이터 셔플링 (과적합 방지)
- **validation_split**: 자동 분할 대신 수동 분할 사용
- **subset**: 특정 데이터 분할만 로드


In [55]:
def load_imdb_datasets(batch_size=32, max_length=None):
    """
    IMDB 데이터셋을 로드하고 기본 전처리를 수행하는 함수
    
    Args:
        batch_size (int): 배치 크기 (기본: 32)
        max_length (int): 최대 시퀀스 길이 (기본: None)
    
    Returns:
        tuple: (train_ds, val_ds, test_ds) 또는 None
    """
    
    base_dir = "../../data/aclImdb"
    
    print("📂 IMDB 데이터셋 로딩 시작")
    print("=" * 50)
    print(f"📊 배치 크기: {batch_size}")
    print(f"📁 베이스 디렉토리: {base_dir}")
    
    # 데이터 디렉토리 존재 확인
    directories = {
        'train': f"{base_dir}/train",
        'val': f"{base_dir}/val", 
        'test': f"{base_dir}/test"
    }
    
    for name, path in directories.items():
        if not os.path.exists(path):
            print(f"❌ {name} 디렉토리가 존재하지 않습니다: {path}")
            return None, None, None
    
    try:
        # 훈련 데이터셋 로드
        print("\n🔄 훈련 데이터셋 로딩 중...")
        train_ds = keras.utils.text_dataset_from_directory(
            directories['train'],
            batch_size=batch_size,
            shuffle=True,           # 훈련 데이터는 셔플
            seed=1337,             # 재현 가능성을 위한 시드
            max_length=max_length   # 최대 길이 제한 (옵션)
        )
        
        # 검증 데이터셋 로드  
        print("🔄 검증 데이터셋 로딩 중...")
        val_ds = keras.utils.text_dataset_from_directory(
            directories['val'],
            batch_size=batch_size,
            shuffle=False,          # 검증 데이터는 셔플하지 않음
            max_length=max_length
        )
        
        # 테스트 데이터셋 로드
        print("🔄 테스트 데이터셋 로딩 중...")
        test_ds = keras.utils.text_dataset_from_directory(
            directories['test'],
            batch_size=batch_size,
            shuffle=False,          # 테스트 데이터는 셔플하지 않음
            max_length=max_length
        )
        
        print("\n✅ 모든 데이터셋 로딩 완료!")
        
        # 데이터셋 정보 확인
        print("\n📊 데이터셋 정보:")
        print("-" * 30)
        
        datasets = [
            ("TRAIN", train_ds),
            ("VALIDATION", val_ds), 
            ("TEST", test_ds)
        ]
        
        for name, dataset in datasets:
            # 배치 수 계산 (근사치)
            batch_count = 0
            total_samples = 0
            
            for batch in dataset.take(5):  # 처음 5개 배치만 확인
                batch_count += 1
                total_samples += len(batch[0])
            
            # 전체 배치 수 추정
            estimated_batches = len(list(dataset))
            estimated_samples = estimated_batches * batch_size
            
            print(f"  {name:>10}: ~{estimated_samples:,}개 샘플 ({estimated_batches:,} 배치)")
        
        # 클래스 정보 확인
        class_names = train_ds.class_names
        print(f"\n🏷️ 클래스 정보: {class_names}")
        print(f"   - {class_names[0]} → 라벨 0")
        print(f"   - {class_names[1]} → 라벨 1")
        
        return train_ds, val_ds, test_ds
        
    except Exception as e:
        print(f"❌ 데이터셋 로딩 중 오류 발생: {e}")
        return None, None, None

# 데이터셋 로딩 실행
BATCH_SIZE = 32
train_ds, val_ds, test_ds = load_imdb_datasets(batch_size=BATCH_SIZE)

if train_ds is not None:
    print("\n🎉 데이터셋 준비 완료!")
    print(f"💾 메모리 사용량 최적화를 위해 배치 크기 {BATCH_SIZE} 사용")
else:
    print("\n💥 데이터셋 로딩 실패!")
    print("💡 데이터 다운로드 및 분할을 먼저 실행해주세요.")

📂 IMDB 데이터셋 로딩 시작
📊 배치 크기: 32
📁 베이스 디렉토리: ../../data/aclImdb

🔄 훈련 데이터셋 로딩 중...
Found 20000 files belonging to 2 classes.
🔄 검증 데이터셋 로딩 중...
Found 5000 files belonging to 2 classes.
🔄 테스트 데이터셋 로딩 중...
Found 25000 files belonging to 2 classes.

✅ 모든 데이터셋 로딩 완료!

📊 데이터셋 정보:
------------------------------
       TRAIN: ~20,000개 샘플 (625 배치)
  VALIDATION: ~5,024개 샘플 (157 배치)
        TEST: ~25,024개 샘플 (782 배치)

🏷️ 클래스 정보: ['neg', 'pos']
   - neg → 라벨 0
   - pos → 라벨 1

🎉 데이터셋 준비 완료!
💾 메모리 사용량 최적화를 위해 배치 크기 32 사용


## 3.2 데이터 구조 분석

### 🔍 원시 데이터 특성 파악
실제 텍스트 데이터의 구조와 특성을 분석하여 전처리 전략을 수립합니다.

### 📊 분석 항목
- **데이터 타입**: 문자열 텐서 확인
- **배치 구조**: (텍스트, 라벨) 쌍 검증  
- **텍스트 길이**: 최소/최대/평균 길이 분석
- **라벨 분포**: 클래스 불균형 확인
- **실제 내용**: 샘플 리뷰 내용 검토


In [57]:
def analyze_raw_data(dataset, dataset_name="DATASET", max_samples=3):
    """
    원시 텍스트 데이터의 구조와 특성을 상세 분석하는 함수
    
    Args:
        dataset: 분석할 TensorFlow 데이터셋
        dataset_name: 데이터셋 이름 (출력용)
        max_samples: 분석할 최대 샘플 수
    """
    
    if dataset is None:
        print(f"❌ {dataset_name} 데이터셋이 없습니다.")
        return
    
    print(f"🔍 {dataset_name} 데이터 구조 분석")
    print("=" * 50)
    
    # 첫 번째 배치 가져오기
    sample_batch = next(iter(dataset))
    texts, labels = sample_batch
    
    # 기본 구조 정보
    print(f"📊 배치 정보:")
    print(f"  텍스트 형태: {texts.shape} (배치크기={texts.shape[0]})")
    print(f"  텍스트 타입: {texts.dtype}")
    print(f"  라벨 형태: {labels.shape}")
    print(f"  라벨 타입: {labels.dtype}")
    
    # 라벨 분포 확인
    unique_labels, _, counts = tf.unique_with_counts(labels)
    print(f"\n🏷️ 현재 배치의 라벨 분포:")
    for label, count in zip(unique_labels.numpy(), counts.numpy()):
        label_name = "긍정" if label == 1 else "부정"
        print(f"  라벨 {label} ({label_name}): {count}개")
    
    # 텍스트 길이 분석
    text_lengths = [len(text.numpy().decode('utf-8').split()) for text in texts[:max_samples]]
    
    print(f"\n📏 텍스트 길이 분석 (처음 {max_samples}개 샘플):")
    for i, length in enumerate(text_lengths):
        print(f"  샘플 {i+1}: {length}단어")
    
    # 샘플 텍스트 내용 확인
    print(f"\n📝 샘플 텍스트 내용:")
    print("-" * 50)
    
    for i in range(min(max_samples, len(texts))):
        text_content = texts[i].numpy().decode('utf-8')
        label_value = labels[i].numpy()
        label_name = "😊 긍정" if label_value == 1 else "😞 부정"
        
        # 텍스트 길이와 요약
        words = text_content.split()
        word_count = len(words)
        
        print(f"\n🎬 샘플 {i+1}: {label_name} (라벨: {label_value})")
        print(f"📊 길이: {word_count}단어, {len(text_content)}자")
        
        # 텍스트 미리보기 (처음 200자)
        preview = text_content[:200]
        if len(text_content) > 200:
            preview += "..."
        
        print(f"📖 내용: {preview}")
        
        # HTML 태그 확인
        if '<br />' in text_content or '<' in text_content:
            print("⚠️ HTML 태그 발견 - 전처리 필요")

# 훈련 데이터 분석
if train_ds is not None:
    analyze_raw_data(train_ds, "TRAIN", max_samples=2)
    
    print("\n" + "="*70)
    print("💡 데이터 분석 결과 요약")
    print("="*70)
    print("✅ 데이터 형태: 문자열 텐서 (string tensor)")
    print("✅ 라벨 인코딩: 0=부정, 1=긍정")
    print("⚠️ HTML 태그 존재: 전처리에서 제거 필요")
    print("📏 텍스트 길이: 다양함 (벡터화에서 표준화 필요)")
    print("🎯 다음 단계: TextVectorization으로 수치화")
else:
    print("❌ 훈련 데이터셋을 먼저 로드해주세요.")


🔍 TRAIN 데이터 구조 분석
📊 배치 정보:
  텍스트 형태: (32,) (배치크기=32)
  텍스트 타입: <dtype: 'string'>
  라벨 형태: (32,)
  라벨 타입: <dtype: 'int32'>

🏷️ 현재 배치의 라벨 분포:
  라벨 0 (부정): 15개
  라벨 1 (긍정): 17개

📏 텍스트 길이 분석 (처음 2개 샘플):
  샘플 1: 871단어
  샘플 2: 140단어

📝 샘플 텍스트 내용:
--------------------------------------------------

🎬 샘플 1: 😞 부정 (라벨: 0)
📊 길이: 871단어, 4771자
📖 내용: I just got the UK 4-disc special edition of Superman 1 for about $5. The additional stuff includes the 1951 feature Superman and the Mole-Men. So I slapped it into the DVD player last night, and here ...
⚠️ HTML 태그 발견 - 전처리 필요

🎬 샘플 2: 😞 부정 (라벨: 0)
📊 길이: 140단어, 763자
📖 내용: There is one good thing in this movie: Lola Glaudini's ass! Sorry to be so blunt but it's the truth. Too bad she didn't do a nude. It would at least have made this mess tolerable. We see another chick...

💡 데이터 분석 결과 요약
✅ 데이터 형태: 문자열 텐서 (string tensor)
✅ 라벨 인코딩: 0=부정, 1=긍정
⚠️ HTML 태그 존재: 전처리에서 제거 필요
📏 텍스트 길이: 다양함 (벡터화에서 표준화 필요)
🎯 다음 단계: TextVectorization으로 수치화


## 7. 텍스트 벡터화 설정

TextVectorization을 사용하여 텍스트를 수치 벡터로 변환합니다.
- Multi-hot encoding (Bag of Words) 방식 사용
- 상위 20,000개 단어만 사용


In [58]:
# 텍스트 벡터화 레이어 생성
text_vectorization = TextVectorization(
    max_tokens=20000,  # 자주 사용하는 단어 20,000개만 지정
    output_mode="multi_hot"  # Multi-hot encoding (BoW 방식)
    # 각 리뷰마다 20,000개 요소를 갖는 배열 생성
    # 배열에서 문장 중 단어가 있는 곳은 1, 없으면 0
)

# 훈련 데이터에서 텍스트만 추출 (레이블 제거)
text_only_train_ds = train_ds.map(lambda x, y: x)

# 어휘 사전 구축
text_vectorization.adapt(text_only_train_ds)

print("텍스트 벡터화 준비 완료")


텍스트 벡터화 준비 완료


## 8. 데이터셋 벡터화

각 데이터셋에 텍스트 벡터화를 적용합니다.
- 유니그램(Unigram): 단어를 1개씩 처리하는 방식
- 멀티프로세싱으로 성능 향상


In [59]:
# 멀티프로세싱으로 CPU 코어 4개 사용하여 벡터화 수행
binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=4
)

binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=4
)

binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=4
)

print("데이터셋 벡터화 완료")


데이터셋 벡터화 완료


## 9. 벡터화 결과 확인

벡터화된 데이터의 형태를 확인합니다.


In [60]:
print("벡터화 후 데이터 확인")
print("=" * 50)

for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[:3])
    print("targets[0]:", targets[:3])
    break


벡터화 후 데이터 확인
inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(
[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]], shape=(3, 20000), dtype=float32)
targets[0]: tf.Tensor([0 0 1], shape=(3,), dtype=int32)


## 10. 모델 정의

간단한 Dense 신경망 모델을 생성합니다.
- 입력층: 20,000차원 (어휘 사전 크기)
- 은닉층: 16개 뉴런, ReLU 활성화
- 드롭아웃: 0.5 (과적합 방지)
- 출력층: 1개 뉴런, 시그모이드 활성화 (이진 분류)


In [61]:
def getModel(max_tokens=20000, hidden_dim=16):
    """모델 생성 및 반환 함수"""
    inputs = keras.Input(shape=(max_tokens,))  # 입력층
    x = layers.Dense(hidden_dim, activation='relu')(inputs)  # 은닉층
    x = layers.Dropout(0.5)(x)  # 드롭아웃
    outputs = layers.Dense(1, activation='sigmoid')(x)  # 출력층
    
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer="rmsprop",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    
    return model

# 모델 생성
model = getModel()
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 11. 모델 훈련

캐싱을 사용하여 효율적으로 모델을 훈련합니다.
- cache(): 첫 번째 에포크에서 전처리를 한 번만 수행하고 메모리에 캐싱
- 메모리에 들어갈 만큼 작은 데이터셋일 때만 효과적


In [63]:
# 콜백 설정: 최고 성능 모델 저장
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "binary_1gram.keras", 
        save_best_only=True
    )
]

# 모델 훈련
model.fit(
    binary_1gram_train_ds.cache(),  # 훈련 데이터 (캐싱 적용)
    validation_data=binary_1gram_val_ds,  # 검증 데이터
    epochs=10,  # 에포크 수
    callbacks=callbacks  # 콜백
)

print("모델 훈련 완료")


Epoch 1/10
625/625 [==============================] - 7s 11ms/step - loss: 0.2973 - accuracy: 0.8869 - val_loss: 0.2946 - val_accuracy: 0.8814
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2532 - accuracy: 0.9104 - val_loss: 0.3049 - val_accuracy: 0.8828
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2242 - accuracy: 0.9205 - val_loss: 0.3155 - val_accuracy: 0.8860
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2232 - accuracy: 0.9230 - val_loss: 0.3225 - val_accuracy: 0.8870
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2118 - accuracy: 0.9279 - val_loss: 0.3545 - val_accuracy: 0.8760
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2067 - accuracy: 0.9337 - val_loss: 0.3493 - val_accuracy: 0.8886
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2074 - accuracy: 0.9331 - val_loss: 0.3645 - val_accuracy: 0.8818
Epoch

## 12. 모델 평가

저장된 최고 성능 모델을 로드하여 테스트 데이터로 평가합니다.


In [64]:
# 저장된 모델 로드
model = models.load_model("binary_1gram.keras")

# 테스트 데이터로 평가
test_results = model.evaluate(binary_1gram_test_ds)
print(f"테스트셋 정확도: {test_results[1]:.4f}")


782/782 [==============================] - 6s 7ms/step - loss: 0.3027 - accuracy: 0.8742
테스트셋 정확도: 0.8742


# Part V: 프로젝트 결론 및 실무 적용

## 5.1 구현 성과 요약

### 🎯 달성한 목표
1. **완전한 NLP 파이프라인 구축**
   - ✅ 대용량 데이터 다운로드 및 전처리 자동화
   - ✅ 체계적인 데이터 분할 (train/val/test)
   - ✅ 효율적인 텍스트 벡터화 (Bag of Words)
   - ✅ 신경망 모델 구축 및 훈련

2. **실무 수준의 코드 품질**
   - ✅ 오류 처리 및 예외 상황 대응
   - ✅ 진행률 표시 및 상태 모니터링
   - ✅ 재현 가능한 결과 보장 (시드 설정)
   - ✅ 모듈화된 함수 설계

3. **성능 최적화 기법 적용**
   - ✅ 데이터 캐싱으로 훈련 속도 향상
   - ✅ 멀티프로세싱 활용
   - ✅ 배치 처리 최적화
   - ✅ 메모리 효율적 데이터 로딩

## 5.2 기술적 성과

### 📊 모델 성능 지표
```
예상 성능 (IMDB 벤치마크 기준):
- Baseline (Random): 50% 정확도
- Bag of Words + Dense: 85-89% 정확도  
- 고급 모델 (RNN/BERT): 90-95% 정확도
```

### 🛠️ 사용된 핵심 기술
- **TensorFlow/Keras**: 딥러닝 프레임워크
- **TextVectorization**: 효율적인 텍스트 전처리
- **tf.data API**: 고성능 데이터 파이프라인
- **Multi-hot Encoding**: 해석 가능한 특성 표현

## 5.3 실무 적용 방안

### 🏢 산업별 활용 사례

#### 1) 전자상거래
```python
# 제품 리뷰 자동 분석
review_sentiment = model.predict("This product is amazing!")
if review_sentiment > 0.5:
    update_product_rating(positive=True)
```

#### 2) 소셜 미디어 모니터링
```python
# 브랜드 멘션 감성 분석
brand_mentions = collect_social_posts("brand_name")
sentiment_scores = model.predict(brand_mentions)
generate_sentiment_report(sentiment_scores)
```

#### 3) 고객 서비스
```python
# 문의 내용 우선순위 분류
inquiry_sentiment = model.predict(customer_message)
if inquiry_sentiment < 0.3:  # 매우 부정적
    escalate_to_manager(inquiry_id)
```

### 🚀 확장 가능한 아키텍처

#### 프로덕션 배포 구조
```
사용자 입력 → API Gateway → 
전처리 서버 → ML 모델 서버 → 
결과 캐싱 → 클라이언트 응답
```

#### 모델 업데이트 전략
```
1. 새로운 데이터 수집
2. 자동 재훈련 파이프라인
3. A/B 테스트로 성능 검증  
4. 점진적 모델 배포
```

## 5.4 개선 방향 및 차세대 기법

### 📈 단계적 성능 개선 로드맵

#### Phase 1: 기본 성능 향상 (즉시 적용 가능)
- **TF-IDF 벡터화**: 단어 중요도 가중치 적용
- **N-gram 확장**: 단어 조합 패턴 인식
- **정규화 강화**: Dropout, Batch Normalization
- **하이퍼파라미터 튜닝**: 학습률, 배치 크기 최적화

#### Phase 2: 고급 모델 도입 (중기 목표)
- **임베딩 레이어**: 밀집 벡터 표현 학습
- **순환 신경망 (RNN/LSTM)**: 순서 정보 활용
- **합성곱 신경망 (CNN)**: 지역적 패턴 인식
- **어텐션 메커니즘**: 중요 단어 집중

#### Phase 3: 최신 기법 적용 (장기 목표)
- **사전 훈련 모델**: BERT, RoBERTa, GPT
- **전이 학습**: 도메인 특화 파인튜닝
- **멀티모달**: 텍스트 + 이미지 + 메타데이터
- **설명 가능한 AI**: 모델 결정 근거 제시

### 🔬 실험적 접근법

#### 데이터 증강 기법
```python
# 동의어 치환으로 데이터 확장
augmented_texts = synonym_replacement(original_texts)

# 백번역으로 다양성 증가  
backtranslated = translate(texts, 'en->ko->en')
```

#### 앙상블 학습
```python
# 다양한 모델 조합으로 성능 향상
models = [bow_model, tfidf_model, embedding_model]
ensemble_prediction = weighted_average(models, weights)
```

## 5.5 학습 성과 및 다음 단계

### 🎓 획득한 핵심 역량
1. **자연어 처리 파이프라인**: 텍스트 데이터 → 예측 결과
2. **대용량 데이터 처리**: 효율적인 I/O 및 메모리 관리
3. **실무 코딩 스킬**: 오류 처리, 로깅, 모니터링
4. **모델 평가 및 개선**: 성능 측정 및 최적화 전략

### 🚦 다음 학습 권장 사항

#### 심화 학습 주제
1. **고급 NLP 기법**
   - Transformer 아키텍처 이해
   - BERT, GPT 활용법
   - 한국어 NLP 특화 기법

2. **MLOps 및 배포**
   - Docker 컨테이너화
   - Kubernetes 오케스트레이션
   - CI/CD 파이프라인 구축

3. **성능 최적화**
   - 모델 압축 및 양자화
   - 추론 속도 최적화
   - 분산 학습 시스템

### 🎯 실습 프로젝트 제안
1. **다른 데이터셋 적용**: 한국어 리뷰, 뉴스 분류
2. **실시간 시스템 구축**: 웹 인터페이스 + API 서버
3. **비교 연구**: 다양한 모델 성능 벤치마킹
4. **도메인 특화**: 의료, 금융, 법률 텍스트 분석

---

## 📚 추천 자료 및 참고 문헌

### 핵심 논문
- Maas et al. (2011): "Learning Word Vectors for Sentiment Analysis"
- Mikolov et al. (2013): "Efficient Estimation of Word Representations"
- Vaswani et al. (2017): "Attention Is All You Need"

### 실무 자료
- TensorFlow Text Guide: https://www.tensorflow.org/text
- Hugging Face Transformers: https://huggingface.co/transformers
- Papers With Code NLP: https://paperswithcode.com/area/natural-language-processing

### 한국어 NLP 자료
- KoBERT, KoGPT: 한국어 사전 훈련 모델
- 한국어 Embedding: FastText, Word2Vec
- 형태소 분석기: KoNLPy, Mecab

---

**🎉 축하합니다! 완전한 NLP 감성 분석 시스템을 구축했습니다!**

이제 여러분은 실무에서 바로 활용할 수 있는 텍스트 분석 역량을 갖추었습니다. 
계속해서 새로운 기법을 학습하고 실제 프로젝트에 적용해보세요! 🚀
